From Statistics Denmark's api console, we obtain two data set dealing with unemployment.

The first data set holds data from 1992 to 2006. The title is "AAR: Unemployed by area and gender (year) (DISCONTINUED)". From the data set we extract the municipality code, year, and unemployment data. Sources: https://www.statistikbanken.dk/statbank5a/SelectVarVal/Define.asp?MainTable=AAR&PLanguage=0&PXSId=0&wsid=cftree Found 18-08-2023. 

The second data set holds data from 2007 to 2022. The title is "AUL01: Full-time unemployed by area, benefit type, social security fund, age and gender." From the data set we extract the municipality code, year, and unemployment data. Source: https://www.statistikbanken.dk/statbank5a/SelectVarVal/Define.asp?MainTable=AUL01&TabStrip=Select&PLanguage=0&FF=20. Found 18-08-2023.

We load a csv file with old and new municipality codes, which we use to create consistency in the data for the purpose of analysis.  


Script by Lars Kjær

In [35]:
# Get the first data set from api.statbank
import pandas as pd
url1 = 'https://api.statbank.dk/v1/data/AAR/CSV?valuePresentation=Code&OMR%C3%85DE=*&K%C3%98N=*&Tid=*'
aar_raw = pd.read_csv(url1, sep = ";")

In [36]:
# rename columns
aar_raw.columns = ['old_muni_code', 'gender', 'year', 'unemployed']
aar_raw = aar_raw[['old_muni_code', 'year', 'unemployed']]

# subset on year
aar_1992_ = aar_raw.query('year >= 1992').reset_index(drop=True) 

# delete rows with unwished value
aar_1992_.drop(aar_1992_[aar_1992_['old_muni_code']== 'L1'] .index, inplace = True)
aar_1992_['old_muni_code']  = aar_1992_['old_muni_code'].astype(int)


# get new and old municipality codes 
new_old_muni_code = pd.read_csv('nye_og_gamle_kommuner.csv')

aar_1992_old_new_muni_codes = pd.merge(aar_1992_, new_old_muni_code, how='left', on='old_muni_code')
aar_1992_added_new_muni_codes = aar_1992_old_new_muni_codes[['muni_code', 'year', 'unemployed']]

# Aggregate group
aar_1992_added_new_muni_codes = aar_1992_added_new_muni_codes.groupby(['muni_code', 'year'])['unemployed'].agg('sum').to_frame().reset_index()
aar_1992_added_new_muni_codes['muni_code'] = aar_1992_added_new_muni_codes['muni_code'].astype(int)

In [37]:
# Get the second data set from api.statbank
url2 = 'https://api.statbank.dk/v1/data/AUL01/CSV?valuePresentation=Code&YDELSESTYPE=TOT&Tid=*&OMR%C3%85DE=101%2C147%2C151%2C153%2C155%2C157%2C159%2C161%2C163%2C165%2C167%2C169%2C173%2C175%2C183%2C185%2C187%2C190%2C201%2C210%2C217%2C219%2C223%2C230%2C240%2C250%2C253%2C259%2C260%2C265%2C269%2C270%2C306%2C316%2C320%2C326%2C329%2C330%2C336%2C340%2C350%2C360%2C370%2C376%2C390%2C400%2C410%2C420%2C430%2C440%2C450%2C461%2C479%2C480%2C482%2C492%2C510%2C530%2C540%2C550%2C561%2C563%2C573%2C575%2C580%2C607%2C615%2C621%2C630%2C657%2C661%2C665%2C671%2C706%2C707%2C710%2C727%2C730%2C740%2C741%2C746%2C751%2C756%2C760%2C766%2C773%2C779%2C787%2C791%2C810%2C813%2C820%2C825%2C840%2C846%2C849%2C851%2C860'
aul01_raw = pd.read_csv(url2, sep = ";")

In [38]:
# Subset and rename columns
aul01_raw1 = aul01_raw.iloc[:, 1:]
aul01_raw1.columns = ['year', 'muni_code', 'unemployed']
aul01 = aul01_raw1

unemployment_data = pd.concat([aar_1992_added_new_muni_codes, aul01]).reset_index(drop=True)

unemployment_data.to_csv('unemployment_data.csv', encoding='utf-8', index=False)